# Examples

## Single row of turbines

In [3]:
import foxes

# Create model book and add turbine type model:
# The file will be searched in current directory,
# and if not found, taken from static library
mbook = foxes.ModelBook()
mbook.turbine_types["NREL5"] = foxes.models.turbine_types.PCtFile("NREL-5MW-D126-H90.csv")

# create ambient wind conditions (here a single state):
states = foxes.input.states.SingleStateStates(ws=9., wd=270., ti=0.06, rho=1.225)

# create wind farm, here a single row of turbines:
farm = foxes.WindFarm()
foxes.input.farm_layout.add_row(
    farm=farm,
    xy_base=[0., 0.], 
    xy_step=[500., 0.], 
    n_turbines=5,
    turbine_models=["NREL5"]
)

# setup the calculation algorithm:
algo = foxes.algorithms.Downwind(mbook, farm, states, 
                                 wake_models=["Bastankhah_linear_k002"])

# run the calculation:
farm_results = algo.calc_farm()
print("\nResults data:\n", farm_results)

Turbine 0, T0: NREL5
Turbine 1, T1: NREL5
Turbine 2, T2: NREL5
Turbine 3, T3: NREL5
Turbine 4, T4: NREL5

Initializing states 'SingleStateStates'
States data:

<xarray.Dataset>
Dimensions:  (state: 1, turbine: 5)
Dimensions without coordinates: state, turbine
Data variables:
    weight   (state, turbine) float64 1.0 1.0 1.0 1.0 1.0

--------------------------------------------------
  Running Downwind: calc_farm
--------------------------------------------------
  n_states  : 1
  n_turbines: 5
--------------------------------------------------
  states    : SingleStateStates
  rotor     : centre (CentreRotor)
  order     : order_wd (OrderWD)
  controller: basic_ctrl (BasicFarmController)
  partialwks: auto (Mapped)
  wake frame: rotor_wd (RotorWD)
--------------------------------------------------
  wakes:
    0) Bastankhah_linear_k002 (BastankhahWake)
--------------------------------------------------
  turbine models:
    0) NREL5 (PCtFile)
-------------------------------------------